# Detección de Comunidades en Twitter

## Cargar el archivo
Se extrajeron tweets del año 2016 en los que se menciona al BBVA de la solución GNIP

Instalamos la librería necesaria para leer archivos de Excel

In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import community
import time

In [ ]:
df = pd.read_excel('../data/Tweets_BBVA.xlsx')
df.head()

Revisamos los nombres de todas las columnas

In [ ]:
df.info()

Creamos g1 como las relaciones de retuiteo (Se descartan los que no hicieron retuit)

In [ ]:
g1 = df[['preferredUsername', 'actor.preferredUsername']][df['actor.preferredUsername'].notnull()]
g1.head()

Creamos g2 para almacenar las relaciones de mención dentro del tweet (Se descartan los que no mencionan a nadie)

In [ ]:
g2 = df[['preferredUsername', 'user_mention_screen_names']][df['user_mention_screen_names'].notnull()]
g2.head()

Creamos una lista 's' que contiene todos los mencionados desagrupados (uno por línea)

In [ ]:
s = g2['user_mention_screen_names'].str.split(',').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'actor.preferredUsername'

Eliminamos de g2 la columna agrupada

In [ ]:
del g2['user_mention_screen_names']

Agregamos a g2 la nueva columna desagrupada, haciendo join por el campo 'preferredUsername'

In [ ]:
g2=g2.join(s)

Unimos los dos dataframes, eliminando las relaciones duplicadas

In [ ]:
h = pd.concat([g1, g2]).drop_duplicates()
h.info()

Nos ha quedado la lista de pares de usuarios donde el primero referencia al segundo

In [ ]:
h.head()

Transformamos el DataFrame en un Grafo Dirigido

In [ ]:
G = nx.from_pandas_edgelist(h, source = 'preferredUsername', target = 'actor.preferredUsername')
print(nx.info(G))

Crear la función top_nodes que mostrará los valores más altos de un diccionario

In [ ]:
def get_top_nodes(cdict, num=5):
    top_nodes ={}
    for i in range(num):
        top_nodes =dict(
            sorted(cdict.items(), key=lambda x: x[1], reverse=True)[:num]
            )
        return top_nodes

#### Grado

Guardar el grado de cada nodo en un diccionario

In [ ]:
gdeg=G.degree()

In [ ]:
get_top_nodes(dict(gdeg))

#### Degree Centrality

In [ ]:
degree_centrality =nx.degree_centrality(G)
nx.set_node_attributes(G,degree_centrality, 'dc')
get_top_nodes(degree_centrality)

#### Betweenness

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G)
nx.set_node_attributes(G,betweenness_centrality, 'bc')
get_top_nodes(betweenness_centrality)

#### Closeness

In [ ]:
closeness_centrality =nx.closeness_centrality(G)
nx.set_node_attributes(G,closeness_centrality, 'cc')
get_top_nodes(closeness_centrality)

#### PageRank Centrality

In [ ]:
pagerank_centrality =nx.pagerank(G)
nx.set_node_attributes(G, pagerank_centrality, 'pr')
get_top_nodes(pagerank_centrality)

## Identificando las Comunidades

#### Densidad

In [ ]:
nx.density(G)

Definiendo algunas propiedades para las visualizaciones

In [ ]:
H = nx.convert_node_labels_to_integers(G,label_attribute='old_label')

In [ ]:
pos = nx.spring_layout(H, k=0.3)

In [ ]:
from random import randint
colors = []
n = 1000
for i in range(n):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

#### Algoritmo Label Propagation

In [ ]:
cluslprop=community.label_propagation_communities(H)

In [ ]:
color_map_b = ["black"] * nx.number_of_nodes(H)
counter = 0
for c in cluslprop:
    for n in c:
        color_map_b[n] = colors[counter]
    counter = counter + 1
plt.figure(figsize=(80,45))
nx.draw_networkx_edges(H, pos, alpha=.1)
nx.draw_networkx_nodes(H, pos, node_color=color_map_b)
plt.show()

#### Algoritmo Greedy Modularity

In [ ]:
clusgmod = community.greedy_modularity_communities(H)

In [ ]:
color_map_b = ["black"] * nx.number_of_nodes(H)
counter = 0
for c in clusgmod:
    for n in c:
        color_map_b[n] = colors[counter]
    counter = counter + 1
plt.figure(figsize=(80,45))
nx.draw_networkx_edges(H, pos, alpha=.1)
nx.draw_networkx_nodes(H, pos, node_color=color_map_b)
plt.show()

#### Algoritmo Louvain
Ejecutamos el script functions.py para cargar todas las funciones que se necesitan

In [ ]:
%run functions.py

In [ ]:
cluslouv = get_clusters(H,louvain(H))

In [ ]:
plt.figure(figsize=(80,45))
for cluster,nodes in cluslouv.items():
  nx.draw_networkx_nodes(H, pos, nodelist=nodes, node_color=colors[cluster%len(colors)])
nx.draw_networkx_edges(H, pos, alpha=.1)
plt.show()

In [ ]:
cluslouv.items()

Elaborado por Luis Cajachahua bajo licencia MIT (2022)